In [5]:
from langchain_community.graphs import Neo4jGraph
import os
from dotenv import load_dotenv
load_dotenv()
# Warning control
import warnings
warnings.filterwarnings("ignore")

In [6]:
azure_openai_api_key = os.environ["OPENAI_API_KEY"]
azure_openai_endpoint = os.environ["OPENAI_API_BASE"]

In [7]:
from openai import AzureOpenAI

client = AzureOpenAI(
  api_key = azure_openai_api_key,  
  api_version = "2023-07-01-preview",
  azure_endpoint = azure_openai_endpoint
)
def embed_text(text):
    response = client.embeddings.create(
    input = text,
    model= "text-embedding-ada-002"
    )
    return response.data[0].embedding

In [14]:
def embed_text(text):
    response = client.embeddings.create(
    input = text,
    model= "text-embedding-ada-002"
    )
    return response.data[0].embedding

In [12]:
NEO4J_URI = "bolt://localhost:7687"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "moviesampleproject"
NEO4J_DATABASE = 'neo4j'

In [13]:
graph = Neo4jGraph(url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE)

In [15]:
question = "What movies are about love?"

In [16]:
question_embedding = embed_text(question)
question_embedding[:10]

[-0.004219826776534319,
 -0.03029683791100979,
 0.0007639749674126506,
 -0.02761838585138321,
 -0.007965870201587677,
 0.0019409307278692722,
 -0.006576106883585453,
 -0.021617135033011436,
 -0.002027790993452072,
 -0.008066943846642971]

**Perform similarity search**

In [17]:
graph.query("""
    with $question_embedding as question_embedding
    CALL db.index.vector.queryNodes(
        'movie_tagline_embeddings', 
        $top_k, 
        question_embedding
        ) YIELD node AS movie, score
    RETURN movie.title, movie.tagline, score
    """,
    params={
        "question_embedding": question_embedding,
        "top_k": 5
    })

[{'movie.title': 'Heat',
  'movie.tagline': 'A Los Angeles crime saga',
  'score': 0.8910084962844849},
 {'movie.title': 'Grumpier Old Men',
  'movie.tagline': 'Still Yelling. Still Fighting. Still Ready for Love.',
  'score': 0.8878611922264099},
 {'movie.title': 'Tom and Huck',
  'movie.tagline': 'The Original Bad Boys.',
  'score': 0.8836773633956909},
 {'movie.title': 'Sense and Sensibility',
  'movie.tagline': 'Lose your heart and come to your senses.',
  'score': 0.8762467503547668},
 {'movie.title': 'Balto',
  'movie.tagline': 'Part Dog. Part Wolf. All Hero.',
  'score': 0.8755828142166138}]

In [18]:
question = "What movies are about adventure?"
question_embedding = embed_text(question)
question_embedding[:10]

[0.010790598578751087,
 -0.035882920026779175,
 -0.018095795065164566,
 -0.019574841484427452,
 -0.006266649812459946,
 0.024024836719036102,
 -0.019510533660650253,
 -0.03385084122419357,
 0.008649198338389397,
 -0.009723113849759102]

In [19]:
graph.query("""
    with $question_embedding as question_embedding
    CALL db.index.vector.queryNodes(
        'movie_tagline_embeddings', 
        $top_k, 
        question_embedding
        ) YIELD node AS movie, score
    RETURN movie.title, movie.tagline, score
    """,
    params={
        "question_embedding": question_embedding,
        "top_k": 5
    })

[{'movie.title': 'Toy Story',
  'movie.tagline': 'The adventure takes off!',
  'score': 0.9303785562515259},
 {'movie.title': 'Cutthroat Island',
  'movie.tagline': 'The Course Has Been Set. There Is No Turning Back. Prepare Your Weapons. Summon Your Courage. Discover the Adventure of a Lifetime!',
  'score': 0.8933806419372559},
 {'movie.title': 'Tom and Huck',
  'movie.tagline': 'The Original Bad Boys.',
  'score': 0.8925687670707703},
 {'movie.title': 'Jumanji',
  'movie.tagline': 'Roll the dice and unleash the excitement!',
  'score': 0.8908366560935974},
 {'movie.title': 'Heat',
  'movie.tagline': 'A Los Angeles crime saga',
  'score': 0.8869895935058594}]